# Annual Recycled Energy Saved in Singapore
*Learn how much Singapore is saving energy per years by recycling plastics, paper, glass, ferrous and non-ferrous metal.*

In this project, we will clean our data and prepares for data analysis. We will be using [Singapore NEA Energy Savings | Kaggle](https://www.kaggle.com/eminbasturk/singapore-nea-energy-savings) data to analyze the total garbage collection and recycling rate. The material names are different as these data were collected from different resources. We will be added the latest data of 2020 [waste-statistics-and-overall-recycling](https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling) from the website so that we can have the latest statics analysis. We will be finding how much energy we can produce using [Greentumble](https://greentumble.com/how-does-recycling-save-energy/) key information.

We will be using **Recycling statistics** to calculate energy saved every year from 2003 to 2020 based on five waste types, plastics, paper, glass, ferrous and non-ferrous metal.

## Loading Data

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

energy_saved = pd.read_csv('./Data/energy_saved.csv')
waste_03_17 = pd.read_csv('./Data/wastestats.csv')
waste_18_20 = pd.read_csv('./Data/2018_2020_waste.csv')

## Cleaning Data

In [2]:
clean_waste_18_20 = waste_18_20.rename(
    columns={
        "Waste Type": "waste_type",
        "Total Generated ('000 tonnes)": "total_waste_generated_tonne",
        "Total Recycled ('000 tonnes)": "total_waste_recycled_tonne",
        "Year": "year",
    }
)
clean_waste_18_20["total_waste_generated_tonne"] = (
    clean_waste_18_20["total_waste_generated_tonne"] * 1000
)
clean_waste_18_20["total_waste_recycled_tonne"] = (
    clean_waste_18_20["total_waste_recycled_tonne"] * 1000
)


https://greentumble.com/how-does-recycling-save-energy/

In [3]:
energy_saved

,The table gives the amount of energy saved in kilowatt hour (kWh) and the amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram) per waste type,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,1 barrel oil is approximately 159 litres of oil,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,material,Plastic,Glass,Ferrous Metal,Non-Ferrous Metal,Paper
3,energy_saved,5774 Kwh,42 Kwh,642 Kwh,14000 Kwh,4100 kWh
4,crude_oil saved,16 barrels,0.12 barrels,1.8 barrels,40 barrels,11 barrels


- Transpose
- Removed first two columns and first row
- Resetting index
- Renaming the columns

As you can see, we have three columns, material, energy_saved, and crude_oil_saved.

In [4]:
clean_energy_saved = (
    energy_saved.T.iloc[1:, 2:]
    .reset_index(drop=True)
    .rename(columns={2: "material", 3: "energy_saved", 4: "crude_oil_saved"})
)
clean_energy_saved

,material,energy_saved,crude_oil_saved
0,Plastic,5774 Kwh,16 barrels
1,Glass,42 Kwh,0.12 barrels
2,Ferrous Metal,642 Kwh,1.8 barrels
3,Non-Ferrous Metal,14000 Kwh,40 barrels
4,Paper,4100 kWh,11 barrels


In [5]:
clean_waste_03_17 = waste_03_17.loc[
    :,
    [
        "waste_type",
        "total_waste_generated_tonne",
        "total_waste_recycled_tonne",
        "recycling_rate",
        "year",
    ],
]


In [6]:
# clean_waste.iloc[16,2] = 1260000

Let's add recycling rate into our DataFrame as we will be using it later to analysis.

In [7]:
clean_waste_18_20["recycling_rate"] = round(
    clean_waste_18_20["total_waste_recycled_tonne"]
    / clean_waste_18_20["total_waste_generated_tonne"],
    2,
)
clean_waste_18_20.head()

,waste_type,total_waste_generated_tonne,total_waste_recycled_tonne,year,recycling_rate
0,Construction& Demolition,1624000,1618000,2018,1.00
1,Ferrous Metal,1269000,126000,2018,0.10
2,Paper/Cardboard,1054000,586000,2018,0.56
3,Plastics,949000,41000,2018,0.04
4,Food,763000,126000,2018,0.17


## Data Analysis

In [8]:
data = pd.concat([clean_waste_18_20, clean_waste_03_17]).sort_values(by="year")
overall = data[(data["waste_type"] == "Overall") | (data["waste_type"] == "Total")]


fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=overall["year"],
        y=overall["total_waste_generated_tonne"],
        name="Waste Generated",
    )
)

fig.add_trace(
    go.Bar(
        x=overall["year"],
        y=overall["total_waste_recycled_tonne"],
        name="Waste Recycled",
    )
)

fig.show()


In [9]:
data['waste_type'].value_counts()

Glass                                      18
Paper/Cardboard                            18
Textile/Leather                            18
Plastics                                   17
Total                                      15
Scrap Tyres                                13
Used Slag                                  13
Others (stones, ceramics & rubber etc)     12
Horticultural Waste                        11
Food waste                                 11
Non-ferrous Metals                         11
Sludge                                     11
Construction Debris                        11
Wood/Timber                                11
Ferrous Metal                               9
Wood                                        7
Food                                        7
Scrap tyres                                 5
Ferrous metal                               5
Ash & Sludge                                5
Used slag                                   5
Horticultural waste               

In [10]:
data["waste_type"] = data["waste_type"].str.replace(
    "Non-ferrous metal", "Non-Ferrous Metal"
    )
data["waste_type"] = data["waste_type"].str.replace(
    "Non-ferrous metals", "Non-Ferrous Metal"
    )
data["waste_type"] = data["waste_type"].str.replace(
    "Non-Ferrous Metals", "Non-Ferrous Metal"
    )
data["waste_type"] = data["waste_type"].str.replace(
    "Plastics", "Plastic"
    )
data["waste_type"] = data["waste_type"].str.replace(
    "Ferrous metal", "Ferrous Metal"
    )
data["waste_type"] = data["waste_type"].str.replace(
    "Paper/Cardboard", "Paper"
    )


In [11]:
total_data = data.merge(
    clean_energy_saved, how="left", left_on="waste_type", right_on="material"
).dropna()

total_data["energy_saved"] = total_data.loc[:, "energy_saved"].str.replace("kWh", "")

total_data["energy_saved"] = (
    total_data.loc[:, "energy_saved"].str.replace("Kwh", "").astype(int)
)

total_data.head()


,waste_type,total_waste_generated_tonne,total_waste_recycled_tonne,year,recycling_rate,material,energy_saved,crude_oil_saved
2,Glass,65500,6200.0,2003,0.09,Glass,42,0.12 barrels
10,Plastic,579900,39100.0,2003,0.07,Plastic,5774,16 barrels
11,Paper,1084700,466200.0,2003,0.43,Paper,4100,11 barrels
25,Plastic,683100,74100.0,2004,0.11,Plastic,5774,16 barrels
26,Paper,1132100,519900.0,2004,0.46,Paper,4100,11 barrels


In [12]:
total_data["total_energy_saved"] = (
    total_data.loc[:, "total_waste_recycled_tonne"] * total_data.loc[:, "energy_saved"]
)

total_data.head()


,waste_type,total_waste_generated_tonne,total_waste_recycled_tonne,year,recycling_rate,material,energy_saved,crude_oil_saved,total_energy_saved
2,Glass,65500,6200.0,2003,0.09,Glass,42,0.12 barrels,2.604000e+05
10,Plastic,579900,39100.0,2003,0.07,Plastic,5774,16 barrels,2.257634e+08
11,Paper,1084700,466200.0,2003,0.43,Paper,4100,11 barrels,1.911420e+09
25,Plastic,683100,74100.0,2004,0.11,Plastic,5774,16 barrels,4.278534e+08
26,Paper,1132100,519900.0,2004,0.46,Paper,4100,11 barrels,2.131590e+09


## Visualization

In [13]:
total_data.groupby(by=["waste_type"]).mean()[
    "recycling_rate"
    ].to_frame().style.\
    background_gradient(cmap="Pastel1_r", subset=["recycling_rate"])

,recycling_rate
waste_type,
Ferrous Metal,0.900714
Glass,0.166667
Non-Ferrous Metal,0.942857
Paper,0.498333
Plastic,0.086667


I wanted to check our final data for outliners and patterns. We found out that there was anomaly at year 2018 and to figure it out we have to check our dataset.

In [14]:
fig = px.box(total_data, x="year", y="total_waste_recycled_tonne")
fig.update_traces(quartilemethod="exclusive")
fig.show()


In [15]:
total_data[total_data['year']==2018]

,waste_type,total_waste_generated_tonne,total_waste_recycled_tonne,year,recycling_rate,material,energy_saved,crude_oil_saved,total_energy_saved
228,Non-Ferrous Metal,171000,170000.0,2018,0.99,Non-Ferrous Metal,14000,40 barrels,2.380000e+09
235,Plastic,949000,41000.0,2018,0.04,Plastic,5774,16 barrels,2.367340e+08
236,Paper,1054000,586000.0,2018,0.56,Paper,4100,11 barrels,2.402600e+09
237,Ferrous Metal,1269000,126000.0,2018,0.10,Ferrous Metal,642,1.8 barrels,8.089200e+07
239,Glass,64000,12000.0,2018,0.19,Glass,42,0.12 barrels,5.040000e+05


After going through total waste recycled of 2018, we discovered that total waste generated for Ferrous Metal was 126900 tonne but total recycled waste was 126000. As we know the mean recycling rate for Ferrous metal is 90 but it was showing 10 percent which was odd, so I went back to original data on the site and discovered the mistake. We can clearly see in the [PDF](https://www.nea.gov.sg/docs/default-source/our-services/waste-management/waste-recycling-statistics-2016-to-2019.pdf) that entire zero was missing. 

In [16]:
total_data.loc[237, "total_waste_recycled_tonne"] = 1260000
total_data["total_energy_saved"] = total_data.loc[:, "total_waste_recycled_tonne"] * (
    total_data.loc[:, "energy_saved"]
)

fig = px.box(total_data, x="year", y="total_waste_recycled_tonne")
fig.update_traces(quartilemethod="exclusive") 
fig.show()


The Box Plot of total energy saved is all over the place as some of the material produce higher energy kWh per metric tonne.

In [17]:
fig = px.box(total_data, x="year", y="total_energy_saved")
fig.update_traces(quartilemethod="exclusive") 
fig.show()

We can interact more with our data and look for patter in multilevel scatter plot. As we can see total energy saved from paper and plastic have significantly reduce in past few years due to government initiative to control the waste produce.

In [18]:
fig = px.scatter(
    total_data,
    x="year",
    y="total_energy_saved",
    size="total_waste_recycled_tonne",
    color="material",
    size_max=60,
)
fig.show()


In [19]:
total_data.energy_saved.value_counts()

4100     18
5774     18
42       18
642      14
14000     7
Name: energy_saved, dtype: int64

## Energy saved per year

its time to calculate energy saved every year from 2003 to 2020 based on five waste types, plastics, paper, glass, ferrous and non-ferrous metal. 

- Group by per year
- Summarize and extract total energy saved
- Converting it into Pandas dataframe
- Converting `total_energy_saved`from float to integer

In [20]:
annual_energy_savings = pd.DataFrame(
    total_data.groupby(by=["year"]).sum()["total_energy_saved"],
    columns=["total_energy_saved"],
    ).astype({"total_energy_saved": int})


In [21]:
annual_energy_savings["total_energy_saved"] = (
    round(annual_energy_savings["total_energy_saved"] / 1000000, 2)\
    .astype(str) + " GWh"
)
annual_energy_savings.tail()


,total_energy_saved
year,
2016,5043.54 GWh
2017,4802.68 GWh
2018,5828.76 GWh
2019,4606.34 GWh
2020,3598.42 GWh
